In [171]:
from scipy import integrate
import math
import pandas as pd

# Continuous Probability Distribution

In [166]:
class contjointPDF:
    # constructor to initialize given function, range of x and range of y
    def __init__(self, probfunc, rangex, rangey):

        self.probfunction = probfunc
        self.lower_bound_x = rangex[0]
        self.upper_bound_x = rangex[1]
        self.lower_bound_y = rangey[0]
        self.upper_bound_y = rangey[1]
    
    # calculate joint probability
    def jointxy(self, lower_x, upper_x, lower_y, upper_y, func_to_integrate = 1):
        if func_to_integrate == 1:
            return integrate.dblquad(self.probfunction, lower_x, upper_x, lambda x: lower_y, lambda x: upper_y)[0]
        else:
            return integrate.dblquad(func_to_integrate, lower_x, upper_x, lambda x: lower_y, lambda x: upper_y)[0]
    
    #calculate marginal probability of c
    def marginal_x(self,lower_x, upper_x):
        return integrate.dblquad(self.probfunction, lower_x, upper_x, lambda x: self.lower_bound_y, lambda x: self.upper_bound_y)[0]
    
    #calculate marginal probability of y
    def marginal_y(self,lower_y, upper_y):
        return integrate.dblquad(self.probfunction, self.lower_bound_x, self.upper_bound_x, lambda x: lower_y, lambda x: upper_y)[0]

    # caclulate conditional probability of x given y
    def cond_x_given_y(self, upper_x, upper_y):
        # f(x | y ) = f(x,y) / f(y)
        f_y, _ = integrate.quad(self.probfunction, self.lower_bound_x, self.upper_bound_x, args=(upper_y))
        def resultant_func( x):
            return (1 / (f_y)) * self.probfunction(upper_y, x)
        resultant_prob, _ = integrate.quad(resultant_func, self.lower_bound_x, upper_x)
        return resultant_prob

    #calculate conditional probability of y given x
    def cond_y_given_x(self,upper_y, upper_x):
        # f(y | x ) = f(x,y) / f(x)
        f_x, _ = integrate.quad(self.probfunction, self.lower_bound_y, self.upper_bound_y, args=(upper_x))
        def resultant_func( y):
            return (1 / (f_x)) * self.probfunction(y, upper_x)
        resultant_prob, _ = integrate.quad(resultant_func, self.lower_bound_y, upper_y)
        return resultant_prob

    #calculate expectation of x
    def expectation_x(self):
        def multiply_x(y,x): 
            return x*self.probfunction(y,x)
        return self.jointxy(self.lower_bound_x, self.upper_bound_x, self.lower_bound_y, self.upper_bound_y, multiply_x)

    #calculate expectation of y     
    def expectation_y(self): 
        def multiply_y(y,x): 
            return y*self.probfunction(y,x)
        return self.jointxy(self.lower_bound_x, self.upper_bound_x, self.lower_bound_y, self.upper_bound_y, multiply_y)
    
    # calculate conditional expectation of y given x
    def cond_expect_y_given_x(self, x_value):
        # f(y | x ) = f(x,y) / f(x)
        f_x, _ = integrate.quad(self.probfunction, self.lower_bound_y, self.upper_bound_y, args=(x_value))
        def resultant_func( y):
            return (y / (f_x)) * self.probfunction(y, x_value)
        resultant_prob, _ = integrate.quad(resultant_func, self.lower_bound_y, self.upper_bound_y)
        return resultant_prob

    # calculate conditional ecpectation of x given y
    def cond_expect_x_given_y(self, y_value):
         # f(x| y ) = f(x,y) / f(y)
        f_y, _ = integrate.quad(self.probfunction, self.lower_bound_x, self.upper_bound_x, args=(y_value))
        def resultant_func( x):
            return (x / (f_y)) * self.probfunction(y_value, x)
        resultant_prob, _ = integrate.quad(resultant_func, self.lower_bound_x, self.upper_bound_x)
        return resultant_prob

    #variance of x
    def variance_x(self):
        def multiply_x_squared(y,x): 
            return (x**2)*self.probfunction(y,x)
        e_x_squared = self.jointxy(self.lower_bound_x, self.upper_bound_x, self.lower_bound_y, self.upper_bound_y, multiply_x_squared)
        variance_of_x = e_x_squared - (self.expectation_x())**2
        return variance_of_x

    #variance of y
    def variance_y(self): 
        def multiply_y_squared(y,x): 
            return (y**2)*self.probfunction(y,x)
        e_y_squared = self.jointxy(self.lower_bound_x, self.upper_bound_x, self.lower_bound_y, self.upper_bound_y, multiply_y_squared)
        variance_of_y = e_y_squared - (self.expectation_y())**2
        return variance_of_y

    # conditional variance of x given y
    def cond_var_x_given_y(self, y_value):
        #Var(X|Y) = E[X^2|Y] − (E[X|Y])^2
        f_y, _ = integrate.quad(self.probfunction, self.lower_bound_x, self.upper_bound_x, args=(y_value))
        def resultant_func( x):
            return ((x**2) / (f_y)) * self.probfunction(y_value, x)
        e_x_squared_given_y , _ = integrate.quad(resultant_func, self.lower_bound_x, self.upper_bound_x)
        e_x_given_y_squared = ( self.cond_expect_x_given_y( y_value) )**2
        return e_x_squared_given_y - e_x_given_y_squared

    # conditional variance of y given x
    def cond_var_y_given_x(self, x_value):
        #Var(Y|X) = E[Y^2|X] − (E[Y|X])^2
        f_x, _ = integrate.quad(self.probfunction, self.lower_bound_y, self.upper_bound_y, args=(x_value))
        def resultant_func( y):
            return ((y**2) / (f_x)) * self.probfunction(y, x_value)
        e_y_squared_given_x , _ = integrate.quad(resultant_func, self.lower_bound_x, self.upper_bound_x)
        e_y_given_x_squared = ( self.cond_expect_y_given_x( x_value) )**2
        return e_y_squared_given_x - e_y_given_x_squared

    # covariance of x and y
    def covariance(self):
        def multiply_x_y(y,x): 
            return y*x*self.probfunction(y,x)
        e_xy = self.jointxy(self.lower_bound_x, self.upper_bound_x, self.lower_bound_y, self.upper_bound_y, multiply_x_y)
        covariance_xy = e_xy - (self.expectation_x()*self.expectation_y())
        return covariance_xy
    
    # correlation of x and y
    def correlation(self):
        stand_x = math.sqrt(self.variance_x())
        stand_y = math.sqrt(self.variance_y())
        return self.covariance() / (stand_x * stand_y)
    
    def check_independence


In [253]:
def myfunction(y,x):
    return x**2+y

# initialise by giving enquation, range of x and range of y
calc_continuous = contjointPDF(myfunction,  (0,1), (0,0.9))


0.5261921197753822

In [254]:
# find probability 0 < x< 0.5, 0 < y <0.6
calc_continuous.jointxy(0, 0.5, 0, 0.6)

0.11499999999999999

In [255]:
# calculate 0 < x < 1
calc_continuous.marginal_x(0, 0.7)

0.3863999999999999

In [256]:
# calculate 0 < y < 0.3
calc_continuous.marginal_y(0, 0.3)

0.14500000000000002

In [257]:
# prob x < 0.5 given y < 0.6
calc_continuous.cond_x_given_y(0.5, 0.6) 

0.3972868217054263

In [258]:
# prob y < 0.7 given x <0.3
calc_continuous.cond_y_given_x(0.7, 0.3) # 0.7202

0.6337448559670782

In [259]:
# expectation of x
calc_continuous.expectation_x() # 1.1025

0.4275

In [260]:

# expectation of y
calc_continuous.expectation_y() # 0.854

0.37800000000000006

In [261]:
# conditioal expectation of y given x = 0.3
calc_continuous.cond_expect_y_given_x(0.3) # 0.5

0.5750000000000001

In [262]:
# conditional expectation of x given y = 0.75
calc_continuous.cond_expect_x_given_y(0.8) # 0.483

0.5701754385964912

In [263]:
# variance of x 
calc_continuous.variance_x()

0.13224374999999997

In [264]:
#variance y 
calc_continuous.variance_y()

0.10214100000000001

In [265]:
#conditional variance of x given y = 0.5
calc_continuous.cond_var_x_given_y(0.5)

0.04444444444444451

In [266]:
#conditional variance of y given x = 0.4
calc_continuous.cond_var_y_given_x(0.4)

0.23818487662615595

In [267]:
#covariance of x and y
calc_continuous.covariance()

0.06115500000000004

In [268]:
# correlation of x and y
calc_continuous.correlation()

0.5261921197753822

In [287]:
class discretejointPMF:
    
    def __init__(self, prob_table):
        self.prob_table = prob_table
    
    def jointprob(self,x,y):
        return self.prob_table[x][y]
    
    def marginal_x(self,x_value):
        return self.prob_table[x_value].sum()

    def marginal_y(self,y_value):
        return self.prob_table.loc[y_value].sum()

    def cond_y_given_x(self,y,x):
        joint_prob = self.jointprob(x,y)
        marginal_of_x = self.marginal_x(x)
        return joint_prob / marginal_of_x
    
    def cond_x_given_y(self,x,y):
        joint_prob = self.jointprob(x,y)
        marginal_of_y = self.marginal_y(y)
        return joint_prob / marginal_of_y
    
    def expectation_x(self):
        x_values = self.prob_table.columns
        marginal_prob_of_each_x = [self.marginal_x(i) for i in x_values]
        e_x = sum( [ x_values[i]*marginal_prob_of_each_x[i] for i in range(len(x_values))])
        return e_x


    def expectation_y(self):
        y_values = self.prob_table.index
        marginal_prob_of_each_y = [self.marginal_y(i) for i in y_values]
        e_y = sum( [ y_values[i]*marginal_prob_of_each_y[i] for i in range(len(y_values))])
        return e_y

    def cond_expect_y_given_x(self, x):
        # E(Y|X)
        y_values = self.prob_table.index
        prob_y_x = [self.cond_y_given_x(i,x) for i in y_values]
        e_y_given_x = sum([y_values[i] * prob_y_x[i] for i in range(len(y_values)) ])
        return e_y_given_x


    def cond_expect_x_given_y(self, y):
        # E(X|Y)
        x_values = self.prob_table.columns
        prob_x_y = [self.cond_x_given_y(i,y) for i in x_values]
        e_x_given_y = sum([x_values[i] * prob_x_y[i] for i in range(len(x_values)) ])
        return e_x_given_y

    def variance_x(self):
        x_values = self.prob_table.columns
        marginal_prob_of_each_x = [self.marginal_x(i) for i in x_values]
        e_x_squared = sum( [ (x_values[i]**2)*marginal_prob_of_each_x[i] for i in range(len(x_values))])
        return e_x_squared - (self.expectation_x())**2

    def variance_y(self):
        y_values = self.prob_table.index
        marginal_prob_of_each_y = [self.marginal_y(i) for i in y_values]
        e_y_squared = sum( [ (y_values[i]**2)*marginal_prob_of_each_y[i] for i in range(len(y_values))])
        return e_y_squared - (self.expectation_y())**2

    def cond_var_x_given_y(self,y):
        #E(X^2|Y) - (E(X|Y)^2)
        x_values = self.prob_table.columns
        prob_x_y = [self.cond_x_given_y(i,y) for i in x_values]
        e_x_squared_given_y = sum([(x_values[i]**2) * prob_x_y[i] for i in range(len(x_values)) ])
        return e_x_squared_given_y - (self.cond_expect_x_given_y(y)**2)


    def cond_var_y_given_x(self,x):
        y_values = self.prob_table.index
        prob_y_x = [self.cond_y_given_x(i,x) for i in y_values]
        e_y_squared_given_x = sum([(y_values[i]**2) * prob_y_x[i] for i in range(len(y_values)) ])
        return e_y_squared_given_x - (self.cond_expect_y_given_x(x)**2)

    def covariance(self):
        #E[XY] - E[X]E[Y]
        x_values = self.prob_table.columns
        y_values = self.prob_table.index

        xy_multiply_prob = []
        for i in x_values:
            for j in y_values:
                xy_multiply_prob.append(i*j*self.prob_table[i][j])
        exy = sum(xy_multiply_prob)
        return exy - (self.expectation_x()* self.expectation_y())


    def correlation(self):
        # corr = cov(x,y)/ standx*standy
        stand_x = math.sqrt(self.variance_x())
        stand_y = math.sqrt(self.variance_y())
        return self.covariance() / (stand_x*stand_y)

    def check_independence(self, x, y):
        return self.prob_table[x][y] == self.marginal_x(x) * self.marginal_y(y)
    

# Discrete Probability distribution

In [288]:
# Discrete Probability distribution

#measurememnt of length and width of a box are rounded to nearest mm

#Let X denote the length
#Let y denot the width

#The possible values of X are 130, 131, 132
#The possible values of Y are 20, 21
# there are six combinations

prob_table = pd.DataFrame({ 130: [0.12, 0.08],
                131 : [0.42, 0.28],
                132 : [0.06, 0.04]
                })
prob_table = prob_table.rename(index = {0: 20, 1: 21})

prob_table

,130,131,132
20,0.12,0.42,0.06
21,0.08,0.28,0.04


In [292]:
calc_discrete = discretejointPMF(prob_table)
# find the joint probability
calc_discrete.jointprob(130,20)

0.12

In [239]:
# find probability that x = 131
calc_discrete.marginal_x(131)

0.7

In [240]:
# find probability that y = 21
calc_discrete.marginal_y(21)

0.4

In [241]:
# find conditional probability of y =21 given x = 131
calc_discrete.cond_y_given_x(21,131)

0.4000000000000001

In [242]:
# find conditional probability of x =131 given y = 20
calc_discrete.cond_x_given_y(132, 20)

0.09999999999999998

In [243]:
# find expectation of x
calc_discrete.expectation_x()

130.89999999999998

In [244]:
# find expectation of y
calc_discrete.expectation_y()

20.400000000000002

In [245]:
# find conditional expectation of y given x = 130
calc_discrete.cond_expect_y_given_x(130)

20.4

In [246]:
# find conditional expectation of x given y =21
calc_discrete.cond_expect_x_given_y(21)

130.9

In [247]:
# find variance of x
calc_discrete.variance_x()

0.2900000000045111

In [248]:
# find variance of y
calc_discrete.variance_y()

0.23999999999995225

In [249]:
# find conditional variance of x given y =21
calc_discrete.cond_var_x_given_y(21)

0.2900000000008731

In [250]:
# find conditional variance of y given x = 132
calc_discrete.cond_var_y_given_x(132)

0.2400000000000091

In [251]:
# find covariance of x and y
calc_discrete.covariance()

4.547473508864641e-13

In [252]:
# find correlation
calc_discrete.correlation()

1.723715385135353e-12

In [293]:
#check independence
calc_discrete.check_independence(130,21)

False